In [1]:
import polars as pl

In [17]:
mrna = pl.read_csv("mds_data/preprocessed_74/mrna.csv")
mirna = pl.read_csv("mds_data/preprocessed_74/mirna_genes.csv")
circrna = pl.read_csv("mds_data/preprocessed_74/circrna.csv")

y = pl.read_csv("mds_data/preprocessed_74/annotations_74.csv")
y = y.select(
    "SAMPLE_ID",
    pl.col("disease").alias("class") - 1
).filter(pl.col("class") != -1)

In [18]:
y

SAMPLE_ID,class
str,i64
"""N54""",0
"""N58""",0
"""N60""",0
"""N70""",0
"""N82""",0
…,…
"""V806""",1
"""V839""",1
"""V883""",1


In [9]:
mrna

GENE_ID,GENE_NAME,N54,N58,N60,N70,N82,N83,N84,N85,N86,N87,NV1428,NV911,NV912,V1048,V108,V1090,V125,V1279,V1297,V1321,V1337,V1394,V1422,V1441,V1456,V148,V1505,V1528,V1565,V1591,V1592,V1699,V1708,V1742,V1776,…,V1800,V1823,V1834,V1857,V1860,V1874,V1884,V1920,V1921,V2089,V2092,V2110,V2133,V221,V2241,V344,V359,V406,V456,V513,V538,V553,V574,V624,V630,V637,V655,V67,V712,V714,V716,V777,V806,V839,V883,V888,V940
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ENSG00000103184""","""SEC14L5""",3.191832,4.608801,7.319238,3.59649,2.915345,3.635597,4.091334,2.222533,6.186945,4.806552,1.36016,2.834747,3.123205,4.433899,3.475937,3.674835,4.658861,1.526765,4.814422,4.756125,2.165738,2.68199,5.702921,3.575593,0.729479,2.052301,6.297734,5.947491,4.06144,6.812226,4.923035,3.529177,3.783978,5.211992,1.654315,…,0.449068,2.756501,4.46108,5.886969,6.118641,3.35921,6.167761,3.005617,5.756641,1.407714,5.052864,5.565085,6.958192,3.850786,6.998436,3.333786,6.972187,5.133347,1.625164,5.918773,5.011722,2.053266,10.545634,2.316511,4.113956,3.904561,4.615747,5.886462,3.198,2.002693,2.37527,3.556877,5.280105,4.003687,5.715705,4.498388,4.925127
"""ENSG00000119862""","""LGALSL""",5.770634,7.490982,8.510512,6.344374,3.340744,6.209902,6.014609,5.247551,7.617572,7.238006,3.672824,6.090522,5.507773,6.594243,6.150602,7.972545,6.639792,5.576557,8.018569,7.654567,4.980846,6.951803,8.570198,5.11188,6.72435,5.060741,9.34149,8.37682,6.459276,9.114749,6.799813,5.355627,6.616032,6.843535,4.631705,…,2.395112,6.169491,5.944522,8.062168,7.598963,6.816697,7.563591,6.29872,7.122347,3.773391,6.969111,6.363838,6.256439,5.883172,8.491607,5.529371,8.572595,7.52961,6.07641,9.742103,6.625131,4.594296,12.55952,5.86034,6.607329,6.05699,7.609438,7.772362,6.11855,5.643975,5.751237,5.985363,6.700821,6.866797,7.496231,6.098174,6.915828
"""ENSG00000115461""","""IGFBP5""",0.763427,1.555056,4.970032,1.615615,3.616112,4.242216,5.729791,4.657348,8.877614,5.155293,2.10131,3.08074,4.292106,7.593266,2.914579,6.282658,4.339264,4.670051,5.197992,5.835512,3.376842,6.192029,6.576979,3.154008,4.75449,8.679646,3.415391,3.447538,8.286235,6.682436,7.72313,4.516896,3.911929,8.701579,4.425332,…,2.668041,6.253991,6.21013,9.972572,5.114645,4.823566,4.333174,3.610219,4.273105,4.619715,5.887462,6.878568,7.08212,4.711383,5.421461,3.716759,6.197377,2.626837,2.303898,3.462326,4.672774,3.661823,4.087451,4.637354,5.130132,2.251784,3.685351,11.944963,2.727177,2.065015,1.491171,4.570045,6.776773,2.735298,4.402395,5.244266,6.444655
"""ENSG00000173083""","""HPSE""",6.801632,6.97127,8.019021,7.247556,5.145452,6.233161,6.026417,5.486908,6.926837,6.523908,5.022343,5.626738,6.520086,6.450058,7.835582,8.108182,6.963666,6.316595,7.533324,6.895831,6.111237,6.372656,7.762534,6.583929,6.175677,6.263858,8.418517,7.679227,6.837582,8.286752,6.658419,6.066936,6.290731,6.988807,5.746908,…,5.497745,6.169491,6.644426,6.410566,7.327535,6.474571,8.364255,5.867646,7.311571,5.812901,6.976223,7.239013,7.13695,6.727053,7.273022,4.68012,8.21231,6.623071,6.531355,7.864798,7.226556,7.001357,11.856413,5.911117,8.056937,6.102844,6.942952,7.685873,5.961771,5.75211,6.151162,7.321247,7.018297,7.479884,6.986091,6.903422,6.763856
"""ENSG00000275896""","""PRSS2""",5.405518,4.309661,5.347327,6.010802,5.101363,5.469513,5.508558,6.096834,5.71578,5.471286,6.157133,4.314689,3.77163,3.057743,9.901597,2.450216,4.359004,3.917472,9.613035,6.719354,6.966423,6.917732,2.468472,3.473651,1.98593,4.065086,5.157177,3.677088,2.138807,2.110252,4.689501,3.448809,2.788358,1.248523,6.174041,…,11.869002,2.945123,11.724798,5.389423,5.064127,5.344344,8.607309,8.94153,4.751496,5.330383,2.876957,3.755454,4.008087,4.26569,4.017448,3.978114,3.227898,5.961917,3.223281,5.876427,6.681417,0.959869,2.121981,2.441929,3.632501,3.740194,2.5

In [10]:
mirna

GENE_ID,GENE_NAME,N54,N58,N60,N70,N82,N83,N84,N85,N86,N87,NV1428,NV911,NV912,V1048,V108,V1090,V125,V1279,V1297,V1321,V1337,V1394,V1422,V1441,V1456,V148,V1505,V1528,V1565,V1591,V1592,V1699,V1708,V1742,V1776,…,V1800,V1823,V1834,V1857,V1860,V1874,V1884,V1920,V1921,V2089,V2092,V2110,V2133,V221,V2241,V344,V359,V406,V456,V513,V538,V553,V574,V624,V630,V637,V655,V67,V712,V714,V716,V777,V806,V839,V883,V888,V940
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ENSG00000281842""","""MIR1291""",5.527902,5.217694,4.827953,4.746795,5.269447,5.693518,5.950243,5.67247,6.778606,6.109058,6.780273,5.300908,6.736425,6.896025,6.100281,6.08868,3.963385,5.286152,5.639289,3.768205,5.947432,6.331812,6.715616,5.042213,6.560373,4.015212,7.3634,6.74836,6.330458,5.617936,6.083382,6.866457,12.864729,4.277754,6.638636,…,4.549817,6.560356,6.510821,6.10977,12.349844,6.312244,6.499529,13.976171,5.570629,3.95173,7.134531,7.184004,4.432568,7.457196,5.639579,12.786867,3.32115,6.642162,5.058449,6.554265,5.166,5.177108,6.433378,13.547975,5.569913,3.334703,5.05981,4.303219,5.974754,5.779194,6.442973,4.856991,5.254807,5.600627,7.177887,5.102239,6.855459
"""ENSG00000274705""","""MIR486-1""",6.512182,7.449704,7.201118,2.89961,3.224832,4.16326,4.973389,5.036468,5.625014,5.657443,6.300087,5.939615,4.467628,8.328927,5.377266,6.348177,7.61216,7.345061,4.921492,5.280793,5.540167,5.133105,7.214073,4.641697,7.689515,5.688329,8.189637,7.520366,9.773499,10.289848,6.441216,7.985275,6.312941,7.962075,6.418654,…,0.0,7.773986,5.715283,6.838356,7.997671,0.0,6.23968,6.071887,4.85376,3.412648,6.929659,9.034783,7.18149,7.137318,8.139034,0.0,7.90302,6.101325,0.0,7.453974,3.281779,4.922027,8.007168,6.804265,4.599969,3.048089,7.026999,7.005038,4.041983,0.0,0.0,0.0,7.682987,5.600627,9.077453,8.180075,9.702659
"""ENSG00000272543""","""MIR4787""",7.824595,7.862677,8.240453,8.451051,5.991373,8.172876,7.15924,3.335026,7.602921,4.99642,7.427569,5.300908,6.736425,4.159579,7.226268,10.086574,6.546286,8.061463,4.360138,6.09454,5.540167,0.0,4.75622,5.612507,7.55271,4.716039,8.276809,7.107945,0.0,5.360765,9.523341,7.572134,3.815972,4.029519,8.070034,…,6.748279,9.485601,5.715283,4.565996,8.772925,9.488628,7.156608,11.402488,4.85376,2.542136,7.395853,9.034783,6.549571,8.590214,6.944038,6.355244,5.893967,6.206739,8.680976,8.244038,5.479869,7.367824,7.300239,6.804265,7.249172,6.402897,6.953581,4.953624,8.47924,9.439259,9.458095,9.373292,7.682987,5.916596,9.693195,6.945834,7.436262
"""ENSG00000216141""","""MIR941-2""",6.512182,5.692001,5.395884,6.171621,4.703066,6.679512,6.359435,3.335026,3.710151,4.289275,6.560067,4.520743,5.749893,6.896025,6.100281,5.365778,5.319429,5.286152,5.324579,4.714267,6.264677,4.568553,5.471672,4.641697,6.820858,5.185697,4.261916,5.54375,6.330458,5.617936,4.32826,7.403157,6.532739,5.64554,4.867032,…,5.33053,7.291194,7.157133,5.535216,5.795829,5.834061,6.089797,6.800248,5.828595,4.904342,4.292882,6.77227,4.995087,7.718861,7.205118,5.37276,3.32115,5.58012,6.446171,6.363812,5.956029,8.929709,6.653384,5.817116,5.885754,3.334703,5.05981,3.999446,5.567343,4.59141,5.459461,5.259529,6.814357,6.395183,7.497821,5.102239,8.653887
"""ENSG00000283513""","""MIR941-3""",6.732315,5.217694,5.915817,6.409547,5.10419,6.679512,7.15924,3.871528,3.710151,4.289275,6.780273,4.830046,6.585925,6.896025,6.359806,5.365778,5.485334,5.286152,6.116422,5.115499,5.947432,4.568553,6.305147,5.042213,6.560373,5.185697,5.795855,5.54375,6.330458,5.617936,5.291896,7.490118,6.891842,5.812259,6.158989,…,5.96951,7.552692,7.396585,5.535216,6.782785,6.092874,6.089797,6.936574,5.828595,5.307285,4.292882,6.77227,4.995087,7.718861,7.525089,5.37276,3.32115,5.864133,6.036638,6.363812,6.313426,9.139231,7.300239,5.410583,6.554647,3.573705,5.05981,3.999446,6.292087,4.59141,5.866278

In [11]:
circrna

STRAND,circRNA_ID,GENE_ID,GENE_TYPE,GENE_NAME,N54,N58,N60,N70,N82,N83,N84,N85,N86,N87,NV1428,NV911,NV912,V1048,V108,V1090,V125,V1279,V1297,V1321,V1337,V1394,V1422,V1441,V1456,V148,V1505,V1528,V1565,V1591,V1592,V1699,…,V1800,V1823,V1834,V1857,V1860,V1874,V1884,V1920,V1921,V2089,V2092,V2110,V2133,V221,V2241,V344,V359,V406,V456,V513,V538,V553,V574,V624,V630,V637,V655,V67,V712,V714,V716,V777,V806,V839,V883,V888,V940
str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""+""","""hsa_circ_0001387""","""ENSG00000109685""","""protein_coding""","""NSD2""",9.162363,8.455022,9.505942,9.279107,9.681506,9.1846,8.897165,9.678656,10.004854,10.335874,9.285909,8.982992,9.209281,0.0,0.0,7.73943,9.36201,8.230173,9.111807,7.540116,9.529046,6.082523,10.060707,7.321258,10.090399,11.346624,10.016947,8.645445,10.078127,10.295413,8.546742,9.912503,…,8.863755,10.534036,9.375322,9.603068,9.557179,11.650578,9.542619,9.734117,8.643291,11.702746,9.670853,0.0,0.0,7.833636,8.322064,9.55051,10.224354,9.897214,8.409075,9.840056,8.37511,9.905513,8.765797,8.910815,9.313226,0.0,10.292146,10.203798,8.73284,13.105012,9.115767,11.047649,9.110548,10.052834,10.504202,10.061961,9.189208
"""-""","""hsa_circ_0002484""","""ENSG00000196323""","""protein_coding""","""ZBTB44""",11.674858,11.309454,10.867299,11.574937,11.573306,11.767494,11.55758,12.083219,11.512956,11.854521,11.762059,11.746093,11.962091,11.38117,11.908982,11.318191,11.862704,11.962687,11.569103,11.869013,11.775431,11.968326,11.545265,11.560614,11.951936,12.371877,11.685843,11.286272,12.034067,11.4647,11.828677,11.676981,…,11.183203,10.534036,10.859351,10.187412,11.614616,12.376225,11.656607,11.247589,11.340675,12.540804,11.942467,12.227131,11.483612,11.22022,11.961767,11.664506,11.223751,14.25723,11.312283,10.746212,11.034411,11.948767,11.425968,11.265952,11.751947,12.041647,10.739298,11.084594,11.859116,12.895584,9.752268,12.285416,12.077896,11.525895,11.726027,11.168153,10.647066
"""+""","""hsa_circ_0007848""","""ENSG00000270106""","""protein_coding""","""TSNAX-DISC1""",10.440989,10.539337,10.157297,9.279107,9.873931,10.405575,10.011012,10.52587,10.696196,9.75147,10.177927,9.467604,10.667161,11.229227,11.349732,9.319885,11.023474,11.033404,11.10985,11.093719,11.112706,12.476366,10.060707,11.912495,10.21582,10.539684,8.55995,10.450224,10.449792,8.298853,11.003013,9.590949,…,11.811016,0.0,11.958473,9.603068,9.971737,10.410181,10.126936,8.513982,10.225846,10.302917,10.544518,0.0,11.898524,10.828091,10.640381,10.379743,9.983565,10.049066,10.31285,9.425543,10.956447,10.904761,9.180006,10.952936,10.674409,10.627179,11.154125,10.655981,10.43093,10.01227,10.114466,10.222205,10.845686,10.789256,10.241366,10.531072,8.774994
"""-""","""hsa_circ_0002711""","""ENSG00000180530""","""protein_coding""","""NRIP1""",11.160474,11.309454,11.220768,12.433549,11.39649,10.845867,10.217277,10.677775,11.473442,10.598723,11.000508,9.981566,9.793431,9.061404,10.42423,9.904095,10.235482,10.34195,8.114412,13.123467,9.850584,7.653223,10.908103,8.637763,11.437519,9.710363,10.142361,10.343388,12.449019,11.294839,0.0,10.397502,…,11.129788,10.534036,10.052579,11.601676,9.294527,11.535138,8.958623,12.991334,10.810407,8.661486,11.544035,0.0,12.48336,9.998633,9.641285,10.080432,10.331183,11.533617,13.262953,11.362591,10.109031,0.0,10.178761,6.919779,11.504098,0.0,9.418635,10.203798,10.53777,12.143641,12.643904,13.253566,9.524933,7.887655,9.727304,10.448667,11.35721
"""-""","""hsa_circ_0005542""","""ENSG00000068784""","""protein_coding""","""SRBD1""",9.354694,10.911086,10.398111,9.279107,11.332384,9.768735,10.894894,10.52587,10.004854,10.129595,10.062554,9.981566,10.667161,11.582734,9.425279,10.820924,11.023474,10.226565,10.43217,9.119902,10.697886,12.257767,11.438387,11.704958,11.022655,10.931771,10.

In [ ]:
from src.data_preprocessing import FullOmicDataProcessor

FullOmicDataProcessor(
    mrna, 
    y,
    annotation_cols=["GENE_ID", "GENE_NAME"],
    n_features=200,
    output_dir="mds_data/disease_feature_importances/mrna",
).process_data()

# fodm_mirna = FullOmicDataProcessor(mirna, y)
# fodm_circrna = FullOmicDataProcessor(circrna, y)